In [1]:
import tabula as tb
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
import re
import docx
from docx import Document
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = " ".join(str.split())
    str = str.strip('\n')
    str = str.strip(' ')
    return str

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./result/2012-select.csv', index=False, encoding="utf-8")

In [12]:
### code entry
path = "./data/2012-select.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

higher level topic name: Aeronautics Research
topic name: Topic E2 Aeronautics Research Mission Directorate Select Subtopics 
subtopic name: E2.01Air Traffic Management Research and Development
higher level topic name: Science
topic name: Topic E3 Science Mission Directorate Select Subtopics 
subtopic name: E3.01Laser Transmitters and Receivers for Targeted Earth Science Measurements
subtopic name: E3.02Advanced Technology Telescope for Balloon Mission
subtopic name: E3.03Extreme Environments Technology
higher level topic name: Human Exploration and Operations
topic name: Topic E1 Human Exploration and Operations Mission Directorate Select Subtopics 
subtopic name: E1.01High Power Electric Propulsion Systems
subtopic name: E1.02Nano/Micro Satellite Launch Vehicle Technology
subtopic name: E1.03International Space Station Utilization


In [3]:
def handle(bs):
    totalData = []
    ### get overall topic
    regEvens = re.compile('md-taxonomy even.*')
    regOdds = re.compile('md-taxonomy odd.*')
    evens = bs.find_all("li", {"class": regEvens})
    odds = bs.find_all("li", {"class": regOdds})
    offset = 0
    for index in range(3):
        dic = {}
        higher_topic_block = evens[int(index / 2) + offset] if int(index % 2) == 0 else odds[int(index / 2) + offset]
        dic["Higher Topic"] = getHigherTopicName(higher_topic_block)
        regTopicEvens = re.compile('first-topic even.*')
        regTopicOdds = re.compile('first-topic odd.*')
        topic_even = higher_topic_block.find_all("li", {"class": regTopicEvens})
        topic_odd = higher_topic_block.find_all("li", {"class": regTopicOdds})
        for index2 in range(len(topic_even) + len(topic_odd)):
                topic_block = topic_even[int(index2 / 2)] if int(index2 % 2) == 0 else topic_odd[int(index2 / 2)]
                dic["Focus Area"] = getTopicName(topic_block)
                subtopic_even = topic_block.find_all("li", {"class": "second-topic even"})
                subtopic_odd = topic_block.find_all("li", {"class": "second-topic odd"})
                for index3 in range(len(subtopic_even) + len(subtopic_odd)):
                    subtopic_block = subtopic_even[int(index3 / 2)] if int(index3 % 2) == 0 else subtopic_odd[int(index3 / 2)]
                    dic["Subtopic"] = getSubtopicName(subtopic_block)
                    metaDic = getMetaData(subtopic_block)
                    for k in metaDic:
                        dic[k] = metaDic[k]
                    ### get other data
                    whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
                    new_dic = dic.copy()
                    new_dic["Content"] = filterHTMLstr(whole_info_block.get_text())
                    totalData.append(new_dic)
    return totalData

In [11]:
### get higher level topic
def getHigherTopicName(topic_block):
    higher_topic_name = filterHTMLstr(topic_block.find("h2").get_text())
    print("higher level topic name:", higher_topic_name)  # higher level topic name
    return higher_topic_name

### get topic name
def getTopicName(topic_block):
    topic_name = filterHTMLstr(topic_block.find("h4", {"class": "topicTitle"}).get_text()).replace("PDF", "")
    print("topic name:", topic_name)  # subtopic name
    return topic_name

### handle every sub-topic
def getSubtopicName(subtopic_block):
    subtopic_name = filterHTMLstr(subtopic_block.find("p", {"class": "subtopic"}).get_text())
    print("subtopic name:", subtopic_name)  # subtopic name
    return subtopic_name

In [5]:
### get Lead Center, Participating Center(s), Solicitation Year
def getMetaData(subtopic_block):
    dic = {}
    info_group = subtopic_block.find_all("p", {"class": "l-center"})
    for raw_info in info_group:
        info = filterHTMLstr(raw_info.get_text())
        pair = info.split(":")
        dic[pair[0]] = pair[1]
        # print(pair[0], pair[1])
    # print(dic)
    return dic

In [10]:
regEvens = re.compile('md-taxonomy even.*')
regOdds = re.compile('md-taxonomy odd.*')
evens = bs.find_all("li", {"class": regEvens})
odds = bs.find_all("li", {"class": regOdds})
offset = 0
for index in range(3):
    higher_topic_block = evens[int(index / 2) + offset] if int(index % 2) == 0 else odds[int(index / 2) + offset]
    getHigherTopicName(higher_topic_block)
    regTopicEvens = re.compile('first-topic even.*')
    regTopicOdds = re.compile('first-topic odd.*')
    topic_even = higher_topic_block.find_all("li", {"class": regTopicEvens})
    topic_odd = higher_topic_block.find_all("li", {"class": regTopicOdds})
    for index2 in range(len(topic_even) + len(topic_odd)):
            topic_block = topic_even[int(index2 / 2)] if int(index2 % 2) == 0 else topic_odd[int(index2 / 2)]
            getTopicName(topic_block)
            subtopic_even = topic_block.find_all("li", {"class": "second-topic even"})
            subtopic_odd = topic_block.find_all("li", {"class": "second-topic odd"})
            for index3 in range(len(subtopic_even) + len(subtopic_odd)):
                subtopic_block = subtopic_even[int(index3 / 2)] if int(index3 % 2) == 0 else subtopic_odd[int(index3 / 2)]
                getSubtopicName(subtopic_block)

higher level topic name: Aeronautics Research
topic name: Topic E2 Aeronautics Research Mission Directorate Select Subtopics PDF
subtopic name: E2.01Air Traffic Management Research and Development
higher level topic name: Science
topic name: Topic E3 Science Mission Directorate Select Subtopics PDF
subtopic name: E3.01Laser Transmitters and Receivers for Targeted Earth Science Measurements
subtopic name: E3.02Advanced Technology Telescope for Balloon Mission
subtopic name: E3.03Extreme Environments Technology
higher level topic name: Human Exploration and Operations
topic name: Topic E1 Human Exploration and Operations Mission Directorate Select Subtopics PDF
subtopic name: E1.01High Power Electric Propulsion Systems
subtopic name: E1.02Nano/Micro Satellite Launch Vehicle Technology
subtopic name: E1.03International Space Station Utilization
